# Recommendation System

Recommendation System is a large problem with various applications. The main objective is to provide personalized recommendations to users based on their preferences and behavior. There exists two main approaches to solve this problem: Collaborative Filtering and Content-Based Filtering. A third approach exists named Hybrid which combines both approaches.

## 1. Content-Based Filtering

The content-based approach relies on the similarity between items. We measure the similarity between item that the users liked and the items in the dataset to recommend those items that are the most similar. A common similarity measure is the Cosine Similarity.

## 2. Collaborative Filtering

As the name suggests, Collaborative Filtering relies on the behavior of all the users to make recommendations for specific user.

This approach can also be divided into two view:
* Memory-based Collaborative Filtering : we exploit all the interactions of the users to find similar users and recommend items that they have liked.
* Model-based Collaborative Filtering : we train a model to predict the rating that a user would give to an item.




In [97]:
import numpy as np
import pandas as pd
from pandas.core.interchange.dataframe_protocol import DataFrame

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.neighbors import NearestNeighbors

import tensorflow as tf
from sqlalchemy import column
from tensorflow.keras.layers import Embedding, Input, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2


In [2]:
movies_df = pd.read_csv('csv_files/movies.csv')
ratings_df = pd.read_csv('csv_files/ratings.csv')
tags_df = pd.read_csv('csv_files/tags.csv')

# For future example, we fix a user and a movie
my_user_id = 42
film_id = 84

In [105]:
# display(movies_df.info)
# display(ratings_df.info)
# display(tags_df.info)
#ratings_df.head()
movies_df.shape

(87585, 3)

## 1. Content-based

The content-based approach aims to find similarities between items using intrinsic properties of the items.
Based on our dataset, we have only genre of the movies and the year of release which are independent of the users.
A basic similarity measure is the cosine similarity, however as we have a list of genre, the [Otsuka similarity coefficient](https://en.wikipedia.org/wiki/Yanosuke_Otsuka) seams more relevant.

We decided for a first approach to not use the year of release and only use the genre.



In [4]:
# coef of similarity for two list
def ochiai_coef(list_A: list, list_B: list):
    if len(list_A) == 0 or len(list_B) == 0:
        return 0
    intersect = np.intersect1d(list_A, list_B)
    return len(intersect) / (len(list_A) * len(list_B))**0.5

### Preprocessing

In [5]:
ratings_df[ratings_df.userId == my_user_id]

,userId,movieId,rating,timestamp
7132,42,36,3.0,855645897
7133,42,66,4.0,855646278
7134,42,150,4.0,855648714
7135,42,260,3.0,855646059
7136,42,349,4.0,855649174
7137,42,457,4.0,855648928
7138,42,494,3.0,855645897
7139,42,648,4.0,855645808
7140,42,733,4.0,855645897
7141,42,780,3.0,855645808


In [7]:
movies_clean = movies_df.copy()
movies_clean['list_genre'] = pd.Series(movies_df['genres'].str.split('|'))
movies_clean = movies_clean.drop(columns=['genres'])
movies_clean.head()

,movieId,title,list_genre
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II (1995),[Comedy]


In [9]:
ochiai_coef(movies_clean['list_genre'][2], movies_clean['list_genre'][3])

0.8164965809277261

In [12]:
# get the movieId to numpy array
movie_ids = movies_df["movieId"].to_numpy()
# dict to map index of the dataframe to the position of the liste
id_to_idx = {mv_id: i for i, mv_id in enumerate(movie_ids)}

We use the ochiai coefficient to compute the similarity between two movies and give the top 10 similar item

In [13]:
def recommend(film_id: int, k_best: int = 10):
    idx = id_to_idx[film_id]
    similarity_row = np.zeros(len(movie_ids), dtype=float)

    for j in range(len(movie_ids)):
        if j == idx:
            similarity_row[j] = -1
        else:
            similarity_row[j] = ochiai_coef(movies_clean['list_genre'][idx], movies_clean['list_genre'][j])

    # argpartition va arranger les indices pour que le kieme indice soit le kieme plus grand, puis à gauche le plus grand et a droite les plus petits.
    # id_top_k va contenir les indices des kiemes plus grands (non triés)
    id_top_k = np.argpartition(similarity_row, -k_best)[-k_best:]
    # on recupère les indices de max, argsort les tries par ordre croissant
    # et on inverse l'ordre pour un ordre décroissant
    id_top_k = id_top_k[np.argsort(similarity_row[id_top_k])[::-1]]

    return movie_ids[id_top_k], similarity_row[id_top_k]


In [14]:
top_id, score = recommend(film_id)

print(f"Similar movies to {movies_clean[movies_clean.movieId == film_id]['title'].values[0]}")
for m_id in top_id:
    print(f" - {movies_clean[movies_clean.movieId == m_id]['title'].values[0]}")

Similar movies to Last Summer in the Hamptons (1995)
 - Seven Blessings (2023)
 - Shelter in Solitude (2023)
 - Comeback (2023)
 - Persona Non Grata (2021)
 - For Zeko (2022)
 - Spetters (1980)
 - My Dead Dad (2021)
 - Il grande spirito (2019)
 - Learners (2007)
 - Everything Went Fine (2021)


There is a problematics, I need more than several sec (3-4s) to find the recommendation. It is mostly due to the computation for every movie, so more than 80k iterations.
We need to take another approach, using sparse matrix to optimize the computation and having computation only on existing value.

In [15]:
# equivalent à un OHE mais pour plusieurs label
mlb = MultiLabelBinarizer(sparse_output=True)
movie_genre_token = mlb.fit_transform(movies_clean["list_genre"])
print(movie_genre_token)

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 154170 stored elements and shape (87585, 20)>
  Coords	Values
  (0, 2)	1
  (0, 3)	1
  (0, 4)	1
  (0, 5)	1
  (0, 9)	1
  (1, 2)	1
  (1, 4)	1
  (1, 9)	1
  (2, 5)	1
  (2, 15)	1
  (3, 5)	1
  (3, 15)	1
  (3, 8)	1
  (4, 5)	1
  (5, 6)	1
  (5, 17)	1
  (5, 1)	1
  (6, 5)	1
  (6, 15)	1
  (7, 2)	1
  (7, 4)	1
  (8, 1)	1
  (9, 2)	1
  (9, 17)	1
  (9, 1)	1
  :	:
  (87567, 7)	1
  (87568, 8)	1
  (87568, 18)	1
  (87569, 7)	1
  (87570, 7)	1
  (87571, 7)	1
  (87572, 5)	1
  (87572, 1)	1
  (87573, 8)	1
  (87574, 5)	1
  (87574, 8)	1
  (87575, 8)	1
  (87576, 0)	1
  (87577, 17)	1
  (87577, 11)	1
  (87578, 7)	1
  (87579, 8)	1
  (87580, 8)	1
  (87581, 5)	1
  (87581, 8)	1
  (87582, 8)	1
  (87583, 8)	1
  (87584, 2)	1
  (87584, 7)	1
  (87584, 1)	1


In [122]:
import scipy.sparse as sp

row_norms = np.sqrt(movie_genre_token.multiply(movie_genre_token).sum(axis=1)).A1  # ||x_i|| pour tous les films (A1 = 1D)

def norm(array: sp.csr_matrix):
    sum_dot = (array.multiply(array)).sum(axis=1)
    return np.sqrt(sum_dot).A1

def recommend(film_id: int, k: int = 10):
    i = id_to_idx[film_id]
    m_i = movie_genre_token.getrow(i)
    row_norms = norm(movie_genre_token)

    # numerateur: dot(m_i, movie_genre_token.T)
    # sur des lignes de 0 ou de 1, revient à un len(intersect)
    dots = m_i @ movie_genre_token.T
    dots = dots.toarray().ravel()

    # denom: ||xi|| * ||xj||
    denom = (row_norms[i] * row_norms)

    scores = np.divide(dots, denom, out=np.zeros_like(dots, dtype=float), where=denom != 0)

    scores[i] = -1.0  # exclure lui-même

    idx = np.argpartition(scores, -k)[-k:]
    idx = idx[np.argsort(scores[idx])[::-1]]

    return movie_ids[idx], scores[idx]

In [17]:
top_id, score = recommend(film_id)

print(f"Similar movies to {movies_clean[movies_clean.movieId == film_id]['title'].values[0]}")
for m_id in top_id:
    print(f" - {movies_clean[movies_clean.movieId == m_id]['title'].values[0]}")

Similar movies to Last Summer in the Hamptons (1995)
 - Seven Blessings (2023)
 - Shelter in Solitude (2023)
 - Comeback (2023)
 - Persona Non Grata (2021)
 - For Zeko (2022)
 - Spetters (1980)
 - My Dead Dad (2021)
 - Il grande spirito (2019)
 - Learners (2007)
 - Everything Went Fine (2021)


## 2. Collaborative Filtering

Collaborative filtering is an approach where we use the interactions of the users (ratings, tags, etc.) to find similarities and are capable to give recommendations.
As we saif earlier, collaborative filtering can be divided into two main categories: memory-based and model-based.

### 2.A Memory-based Collaborative Filtering

Memory-based collaborative filtering relies on the usage of all the interactions from the users in our memory (database) to give a prediction. Two approaches are possible:
* User-User : we use the interactions of all the users to find users similar and give as recommendation the items those users liked.
* Item-based Collaborative Filtering : we use the result of the interactions to find items similar and give as recommendation the items those users liked.


In [71]:
score_df = pd.merge(ratings_df, movies_clean, on="movieId")

# score_df['year'] = score_df['title'].str.extract(r'\((\d{4})\)\s*$')
# score_df['title'] = score_df['title'].str.replace(r'\s*\(\d{4}\)\s*$', '', regex=True)
score_df = score_df.explode("list_genre")
score_df = score_df.rename(columns={"list_genre": "genre"})
score_df = score_df.drop(columns=["title", "timestamp"])


In [77]:
score_df[score_df.genre == "(no genres listed)"].count()

userId     55498
movieId    55498
rating     55498
genre      55498
dtype: int64

In [72]:
score_df.head()

,userId,movieId,rating,genre
0,1,17,4.0,Drama
0,1,17,4.0,Romance
1,1,25,1.0,Drama
1,1,25,1.0,Romance
2,1,29,2.0,Adventure


In [92]:
popular_score = pd.pivot_table(score_df, index=["userId"], columns=["genre"], values="rating", aggfunc=lambda s: np.where(s > 2.5, 1, -1).sum()).fillna(0)
popular_score['count'] = popular_score.abs().sum(axis=1)
popular_score = popular_score.div(popular_score['count'], axis=0).round(2)
popular_score = popular_score.drop(columns=['count'])

In [96]:
popular_score.shape

(200948, 20)

In [127]:
popular_score.to_pickle('pickle_files/user_preferencies.pkl')

In [102]:
matrice = popular_score.to_numpy(dtype=np.float32)
user_ids = popular_score.index.to_numpy()
pos = { u: i for i, u in enumerate(user_ids)}

NN = NearestNeighbors(n_neighbors=11,  metric='cosine', algorithm='brute')
NN.fit(matrice)

,"n_neighbors n_neighbors: int, default=5Number of neighbors to use by default for :meth:`kneighbors` queries.",11
,"radius radius: float, default=1.0Range of parameter space to use by default for :meth:`radius_neighbors`queries.",1.0
,"algorithm algorithm: {'auto', 'ball_tree', 'kd_tree', 'brute'}, default='auto'Algorithm used to compute the nearest neighbors:- 'ball_tree' will use :class:`BallTree`- 'kd_tree' will use :class:`KDTree`- 'brute' will use a brute-force search.- 'auto' will attempt to decide the most appropriate algorithm based on the values passed to :meth:`fit` method.Note: fitting on sparse input will override the setting ofthis parameter, using brute force.",'brute'
,"leaf_size leaf_size: int, default=30Leaf size passed to BallTree or KDTree. This can affect thespeed of the construction and query, as well as the memoryrequired to store the tree. The optimal value depends on thenature of the problem.",30
,"metric metric: str or callable, default='minkowski'Metric to use for distance computation. Default is ""minkowski"", whichresults in the standard Euclidean distance when p = 2. See thedocumentation of `scipy.spatial.distance`_ andthe metrics listed in:class:`~sklearn.metrics.pairwise.distance_metrics` for valid metricvalues.If metric is ""precomputed"", X is assumed to be a distance matrix andmust be square during fit. X may be a :term:`sparse graph`, in whichcase only ""nonzero"" elements may be considered neighbors.If metric is a callable function, it takes two arrays representing 1Dvectors as inputs and must return one value indicating the distancebetween those vectors. This works for Scipy's metrics, but is lessefficient than passing the metric name as a string.",'cosine'
,"p p: float (positive), default=2Parameter for the Minkowski metric fromsklearn.metrics.pairwise.pairwise_distances. When p = 1, this isequivalent to using manhattan_distance (l1), and euclidean_distance(l2) for p = 2. For arbitrary p, minkowski_distance (l_p) is used.",2
,"metric_params metric_params: dict, default=NoneAdditional keyword arguments for the metric function.",None
,"n_jobs n_jobs: int, default=NoneThe number of parallel jobs to run for neighbors search.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.",None


In [113]:
def top_k_similar_users(user_id: int, k: int = 10) -> pd.DataFrame:
    i = pos[user_id]
    dist, ind = NN.kneighbors(matrice[i:i+1], n_neighbors=k+1)
    dist = dist.ravel()
    ind = ind.ravel()

    mask = ind != i
    ind = ind[mask][:k]
    dist = dist[mask][:k]
    return user_ids[ind]
    return pd.DataFrame({
        "userId": user_ids[ind],
        "cosine_sim": 1.0 - dist
    })

In [114]:
similar_user = top_k_similar_users(my_user_id)
print(similar_user)

[175291 121194  46123  46615  51151 194356   3818 112700  19205 194579]


In [117]:
most_rated = ratings_df[ratings_df["userId"] == my_user_id].sort_values(by=["rating"], ascending=False).head(10)["movieId"].to_numpy()

array([ 912,  924, 1291, 1371, 1221, 1375, 5060, 1376, 1250, 1272])

In [124]:
movie_recommendation = []
for u_id in similar_user:
    most_rated = ratings_df[ratings_df["userId"] == u_id].sort_values(by=["rating"], ascending=False).head(10)["movieId"].to_numpy()
    movie_recommendation.extend(most_rated)

movie_recommendation = list(set(movie_recommendation))
print(movie_recommendation)

[np.int64(4993), np.int64(260), np.int64(48516), np.int64(6), np.int64(904), np.int64(5899), np.int64(2571), np.int64(527), np.int64(2959), np.int64(913), np.int64(2194), np.int64(912), np.int64(1303), np.int64(142488), np.int64(924), np.int64(541), np.int64(296), np.int64(6441), np.int64(2858), np.int64(1193), np.int64(1196), np.int64(1198), np.int64(109487), np.int64(1200), np.int64(47), np.int64(50), np.int64(1203), np.int64(2872), np.int64(1080), np.int64(1210), np.int64(1209), np.int64(1214), np.int64(318), np.int64(5952), np.int64(1221), np.int64(457), np.int64(589), np.int64(593), np.int64(1233), np.int64(474), np.int64(858), np.int64(1246), np.int64(480), np.int64(1250), np.int64(2019), np.int64(356), np.int64(2021), np.int64(2028), np.int64(3052), np.int64(110), np.int64(1262), np.int64(1136), np.int64(7153), np.int64(111), np.int64(5747), np.int64(3062), np.int64(3578)]


In [125]:
for m_id in movie_recommendation:
    print(f" - {movies_clean[movies_clean.movieId == m_id]['title'].values[0]}")

 - Lord of the Rings: The Fellowship of the Ring, The (2001)
 - Star Wars: Episode IV - A New Hope (1977)
 - Departed, The (2006)
 - Heat (1995)
 - Rear Window (1954)
 - Zulu (1964)
 - Matrix, The (1999)
 - Schindler's List (1993)
 - Fight Club (1999)
 - Maltese Falcon, The (1941)
 - Untouchables, The (1987)
 - Casablanca (1942)
 - Man Who Would Be King, The (1975)
 - Spotlight (2015)
 - 2001: A Space Odyssey (1968)
 - Blade Runner (1982)
 - Pulp Fiction (1994)
 - Battle of Britain (1969)
 - American Beauty (1999)
 - One Flew Over the Cuckoo's Nest (1975)
 - Star Wars: Episode V - The Empire Strikes Back (1980)
 - Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
 - Interstellar (2014)
 - Aliens (1986)
 - Seven (a.k.a. Se7en) (1995)
 - Usual Suspects, The (1995)
 - 12 Angry Men (1957)
 - Excalibur (1981)
 - Monty Python's Life of Brian (1979)
 - Star Wars: Episode VI - Return of the Jedi (1983)
 - Once Upon a Time in the West (C'era una volta il West) (1968

### 2.B Model-based Collaborative Filtering

For the model-based collaborative filtering, it is the usage of a model to find recommendations. Predicting rating, find similar users, find similar items, etc. There is no specific approach for this section just the usage of Machine Learning, deep learning, etc.

From the nature of the problem, clustering model is mostly used as kNN.
Another approach popularized with Netflix's open competition is using Matrix Factorization.

